In [ ]:
## CNN on mnist datastets 

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train , y_train) , (x_test , y_test) = mnist.load_data()
print(len(x_train))
print(len(x_test))

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline 
import matplotlib.cm as cm
import numpy as np

## plot first six trainig images
fig = plt.figure(figsize = (20,20))
for i in range(6):
  ax = fig.add_subplot(1,6,i+1,xticks = [] , yticks = [])
  ax.imshow(x_train[i] , cmap = 'gray')
  ax.set_title(str(y_train[i]))

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111)
visualize_input(x_train[0], ax)

In [ ]:
#Reshape (28x28 -> 28x28x1 for CNN input)
x_train = x_train.reshape(x_train.shape[0] , 28,28,1)
x_test = x_test.reshape(x_test.shape[0] , 28,28,1)

In [ ]:
x_train.shape , x_test.shape

In [ ]:
# Normalize pixel values (0-255 → 0-1)
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
## one hot encoding 
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [ ]:
# build the model object
model = Sequential()

# CONV_1: add CONV layer with RELU activation and depth = 32 kernels
model.add(Conv2D(32, kernel_size=(3, 3), padding='same',activation='relu',input_shape=(28,28,1)))
# POOL_1: downsample the image to choose the best features
model.add(MaxPooling2D(pool_size=(2, 2)))

# CONV_2: here we increase the depth to 64
model.add(Conv2D(64, (3, 3),padding='same', activation='relu'))
# POOL_2: more downsampling
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten since too many dimensions, we only want a classification output
model.add(Flatten())

# FC_1: fully connected to get all relevant data
model.add(Dense(64, activation='relu'))

# FC_2: output a softmax to squash the matrix into output probabilities for the 10 classes
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath='model.best.weights.h5', 
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

hist = model.fit(x_train, y_train, batch_size=64, epochs=10,
          validation_data=(x_test, y_test), callbacks=[checkpointer],
          verbose=2, shuffle=True)


In [ ]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [ ]:
# evaluate test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)